In [9]:
import wrf_management.utilities as ut
import importlib
importlib.reload(ut);
import wrf_management.project_global_constants as gc
importlib.reload(gc)
import wrf_management.geogrid as geo
import wrf_management.ungrib as un
import os
import sqlite3 as sq
import pandas as pd
import wrf_management.base_namelists.base_namelists as bn
importlib.reload(bn);
import f90nml

In [65]:
print(gc.RUN_NAME)
job = 'ungrib_press'
file_types = ['press']
real = False

LIST_S_LINKS = [
    'ungrib',
    'ungrib.exe',
    'link_grib.csh',
]

LIST_H_LINKS = [
    'Vtable',
    'env_WRFv4.bash'
]


run_2019_02_20


In [5]:
con = sq.connect(gc.PATH_DB)

In [99]:
importlib.reload(un)
run_row = un.get_run_row()
print(run_row)

job_row = un.get_next_row(job=job)
print(job_row)

job_path = un.getmk_job_path(run_row,job_row,job)
print(job_path)

untar_path = os.path.join(job_path,'untar')
print(untar_path)

conf_path = un.get_conf_path(run_row)
print(conf_path)

type_rows = pd.DataFrame([un.get_type_row(ft,job_row) for ft in file_types])
print(type_rows)

name_list = un.skim_namelist_copy(
    conf_path,job_path,date =job_row.date,prefix=job
)
print(name_list)

if gc.ID=='taito_login':
    un.copy_hard_links(conf_path,job_path,LIST_H_LINKS)
    un.copy_soft_links(gc.PATH_WPS,job_path,LIST_S_LINKS)
    importlib.reload(un)
    un.untar_the_files(type_rows,untar_path)

index                             1
run_name             run_2019_02_20
metgrid_path                geogrid
config_path          run_2019_02_20
data_path       runs/run_2019_02_20
Name: 0, dtype: object
date            2017-12-01 00:00:00
ungrib_surf                       0
ungrib_press                      1
ungrib_lake                       0
Name: 0, dtype: object
/tmp/wrf_management/data_folder/runs/run_2019_02_20/2017_12_01/ungrib_press
/tmp/wrf_management/data_folder/runs/run_2019_02_20/2017_12_01/ungrib_press/untar
/Users/diego/wrf_management/wrf_management/config_dir/run_2019_02_20
                  date  i  downloaded                      name  untarred  \
0  2017-12-01 00:00:00  0           1  cdas1.20171201.pgrbh.tar         0   

    type  
0  press  
OrderedDict([('share', Namelist([('wrf_core', 'ARW'), ('max_dom', 4), ('start_date', ['2017-12-01_00:00:00', '2017-12-01_00:00:00', '2017-12-01_00:00:00', '2017-12-01_00:00:00']), ('end_date', ['2017-12-01_18:00:00', '2017-12-

In [85]:
gc.ID

'mac_diego'

In [95]:
import subprocess as su
res = su.run(['ls','-l'],stdout=su.PIPE,stderr=su.PIPE)

In [97]:
res.returncode

0